imports

In [1]:
import os
import pandas as pd
from HGNN.train import CSV_processor

parameters

In [2]:
dataPath="/raid/elhamod/Fish/Curated5/"
# image_path='Easy'

input_species_csv_fileName = "cleaned_metadata.csv"#"biology_paper_cleaned_all/cleaned_metadata.csv"

output_directory_name="Easy_30" #curated_minimum5images"#"biology_paper_cleaned_30max"
output_cleaned_species_csv_fileName = "cleaned_metadata.csv"

numberOfImagesPerSpecies_min =30 # If None, no min
numberOfImagesPerSpecies_toKeep =30 # If None, keep all

cleanup metadata

In [3]:
# csv_processor = CSV_processor.CSV_processor(os.path.join(dataPath, image_path), '', cleanup=True)

Subset creation

In [4]:
# get metadata file
metadata_csv_df = pd.read_csv(os.path.join(dataPath, input_species_csv_fileName), sep='\t')

# find species that have a specific number of images
grouped_by = metadata_csv_df['scientificName'].value_counts()
if numberOfImagesPerSpecies_min is not None:
    grouped_by = grouped_by[grouped_by > numberOfImagesPerSpecies_min]
scientificNameValues = grouped_by.index.values
thresholded = metadata_csv_df[metadata_csv_df['scientificName'].isin(scientificNameValues)]

# Get first n images from each
if numberOfImagesPerSpecies_toKeep is not None:
    trimmed = thresholded.groupby('scientificName', group_keys=False)
    trimmed = trimmed.apply(lambda x: x.sample(n=numberOfImagesPerSpecies_toKeep) if len(x) >= numberOfImagesPerSpecies_toKeep else x)
#     trimmed = trimmed.head(numberOfImagesPerSpecies_toKeep)
#     trimmed = trimmed.sample(10)
    trimmed.reset_index(drop=True)
else:
    trimmed = thresholded
print('output', trimmed)

# save the file
path = os.path.join(dataPath, output_directory_name)
try:
    os.mkdir(path)
except:
    pass
trimmed.to_csv(os.path.join(path, output_cleaned_species_csv_fileName), sep='\t')


output                    fileName      Family       Genus       scientificName
35     INHS_FISH_105918.JPG   Clupeidae       Alosa  Alosa chrysochloris
45      INHS_FISH_99256.JPG   Clupeidae       Alosa  Alosa chrysochloris
35070    UWZM-F-0000003.JPG   Clupeidae       Alosa  Alosa chrysochloris
28      INHS_FISH_56969.JPG   Clupeidae       Alosa  Alosa chrysochloris
77        INHS_FISH_066.JPG   Clupeidae       Alosa  Alosa chrysochloris
...                     ...         ...         ...                  ...
35051       OSUM0008121.jpg  Cyprinidae  cyprinella  cyprinella whipplei
35059     OSUM0008089_1.jpg  Cyprinidae  cyprinella  cyprinella whipplei
35053       OSUM0008131.jpg  Cyprinidae  cyprinella  cyprinella whipplei
35027       OSUM0008092.jpg  Cyprinidae  cyprinella  cyprinella whipplei
35025     OSUM0008101_1.jpg  Cyprinidae  cyprinella  cyprinella whipplei

[3450 rows x 4 columns]
